In [1]:
from itertools import chain
from pymongo import MongoClient
import pymysql
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def load_Mongo_recipe():
    """
    MongoDB에 있는 모든 레시피를 불러와 리스트 형태로 반환한다.
    """
    # 커넥션 접속 작업
    HOST = 'cluster0.1lslter.mongodb.net'
    USER = 'Sunyoung'
    PASSWORD = 'sun123'
    DATABASE_NAME = 'recipe_DB'
    COLLECTION_NAME = 'recipe_info_v2'
    MONGO_URI = f"mongodb+srv://{USER}:{PASSWORD}@{HOST}/{DATABASE_NAME}?retryWrites=true&w=majority"

    client = MongoClient(MONGO_URI) 
    db = client[DATABASE_NAME] # Connection
    collection = db[COLLECTION_NAME] # Creating table

    ## MongoDB 에서 데이터 불러오기
    data_list = [i for i in collection.find()]

    return data_list

In [3]:
def list_to_dataframe(data_list):
    """
    list를 넣어주면 더블리스트 풀고 df에 추가한다.
    Mongo DB에 있던 중복치 처리는 덤
    """
    # 데이터 프래임 생성
    df = pd.DataFrame(
    {'Recipe_name':[],
    'Hit_num':[],
    'Serves':[],
    'Cooking_time':[],
    'Level':[],
    'Url':[],
    'Ingredients':[]})

    for row in range(len(data_list)):
        # 재료의 더블 리스트롤 풀어준다
        dict_val = data_list[row]['ingredients'].values()
        ingredients = ','.join(s for s in list(chain(*dict_val))) # 리스트를 텍스트로 변환

        # 데이터 프레임에 추가 (id, name, url, ingredient 순서)
        df.loc[row] = [
            data_list[row]['recipe_name'], 
            data_list[row]['hit_num'],
            data_list[row]['serves'],
            data_list[row]['cooking_time'], 
            data_list[row]['level'],
            data_list[row]['url'], 
            ingredients]

    return df

In [4]:
def preprocessing(df):
    """
    깔끔하게 전처리를 완료하는 함수이다.
    """
    df_clean = df.copy()

    # 중복 제거 및 재정렬
    df_clean = df.copy()
    df_clean.drop_duplicates(subset='Url', inplace = True)
    df_clean.reset_index(inplace = True, drop = True)

    # 컬럼 형식 맞추기
    df_clean['Hit_num'] = df_clean['Hit_num'].str.replace(',','').astype(int)
    df_clean['Serves'] = df_clean['Serves'].str.extract(r'(\d+)').astype(int)
    df_clean['Cooking_time'] = df_clean['Cooking_time'].str.replace('2시간','120').str.extract(r'(\d+)').astype(int)

    return df_clean

In [10]:
# 정제된 레시피 목록을 localDB에 저장한다.
def get_connection():

    HOST = 'localhost'
    USER = 'sunyoung'
    PASSWORD = '0720'
    DB_name='recipe'

    conn = pymysql.connect(host = HOST, user = USER, password = PASSWORD, db = DB_name, charset='utf8') 
    return conn

def init_table(conn):
    """
    recipe Table을 초기화
    """
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS recipe;")
    cur.execute("""CREATE TABLE recipe (
        id INTEGER NOT NULL AUTO_INCREMENT PRIMARY KEY,
        Recipe_name VARCHAR(200),
        Hit_num INTEGER,
        Serves INTEGER,
        Cooking_time INTEGER,
        Level VARCHAR(200),
        Url VARCHAR(1000),
        Ingredients VARCHAR(1000));""")
    
    conn.commit()
    pass

def df_to_sql(conn, df_clean):
    """
    레시피 데이터 프레임을 Mysql에 저장
    """
    cur = conn.cursor()
    query = "INSERT INTO recipe(Recipe_name, Hit_num, Serves, Cooking_time, Level, Url, Ingredients) VALUES (%s,%s,%s,%s,%s,%s,%s);"

    for row in range(len(df_clean)):
        data = df_clean.iloc[row, :].tolist()
        cur.execute(query, data)
        print(row,'행 까지 완료')
    conn.commit()
    pass

### Test

In [ ]:
## MongoDB 에서 데이터 불러오기
data_list = load_Mongo_recipe()

# Mongo DB의 데이터를 df로 변환 후 전처리
df_clean = list_to_dataframe(data_list)
df = preprocessing(df_clean)

In [ ]:
# DB 저장
conn = get_connection() # 커넥션 가져옴
init_table(conn) # 초기화
df_to_sql(conn, df)

0 행 까지 완료
1 행 까지 완료
2 행 까지 완료
3 행 까지 완료
4 행 까지 완료
5 행 까지 완료
6 행 까지 완료
7 행 까지 완료
8 행 까지 완료
9 행 까지 완료
10 행 까지 완료
11 행 까지 완료
12 행 까지 완료
13 행 까지 완료
14 행 까지 완료
15 행 까지 완료
16 행 까지 완료
17 행 까지 완료
18 행 까지 완료
19 행 까지 완료
20 행 까지 완료
21 행 까지 완료
22 행 까지 완료
23 행 까지 완료
24 행 까지 완료
25 행 까지 완료
26 행 까지 완료
27 행 까지 완료
28 행 까지 완료
29 행 까지 완료
30 행 까지 완료
31 행 까지 완료
32 행 까지 완료
33 행 까지 완료
34 행 까지 완료
35 행 까지 완료
36 행 까지 완료
37 행 까지 완료
38 행 까지 완료
39 행 까지 완료
40 행 까지 완료
41 행 까지 완료
42 행 까지 완료
43 행 까지 완료
44 행 까지 완료
45 행 까지 완료
46 행 까지 완료
47 행 까지 완료
48 행 까지 완료
49 행 까지 완료
50 행 까지 완료
51 행 까지 완료
52 행 까지 완료
53 행 까지 완료
54 행 까지 완료
55 행 까지 완료
56 행 까지 완료
57 행 까지 완료
58 행 까지 완료
59 행 까지 완료
60 행 까지 완료
61 행 까지 완료
62 행 까지 완료
63 행 까지 완료
64 행 까지 완료
65 행 까지 완료
66 행 까지 완료
67 행 까지 완료
68 행 까지 완료
69 행 까지 완료
70 행 까지 완료
71 행 까지 완료
72 행 까지 완료
73 행 까지 완료
74 행 까지 완료
75 행 까지 완료
76 행 까지 완료
77 행 까지 완료
78 행 까지 완료
79 행 까지 완료
80 행 까지 완료
81 행 까지 완료
82 행 까지 완료
83 행 까지 완료
84 행 까지 완료
85 행 까지 완료
86 행 까지 완료
87 행 까지 완료
88 행 까지 완료
89 행 까지 완료
90 행 까지 완료
91 행 까지 완

DataError: (1406, "Data too long for column 'Ingredients' at row 1")

### 여기 부터 모델링

In [ ]:
def count_tokenizer(text):
    """
    텍스트 전처리를 위한 함수.
    이후 아래 find_sim_recipe 함수에서 CountVectorizer(tokenizer = count_tokenizer) 로 사용한다.
    """
    pass



In [ ]:
def find_sim_recipe(df, my_ingredients_list):
    """
    내가 가진 재료와 비슷한 레시피를 반환한다.
    df : MongoDB로 만든 데이터프래임
    my_ingredients_list : 내가 가진 재료 리스트. 콤마(,)로 구분한 string
    """
    # 내 재료와 레시피 재료 
    my_ingredients = [my_ingredients_list]
    recipe_ingre_list = list(df['Ingredients'])

    doc_list = my_ingredients + recipe_ingre_list
    
    # Embedding
    vect = CountVectorizer(lowercase=False)
    ingredients_mat = vect.fit_transform(doc_list)

    # Cosine similarity
    similarity = cosine_similarity(ingredients_mat[0], ingredients_mat[1:]) # 내 재료, 레시피 재료
    # print(similarity) # 모든 레시피의 유사도를 다 보여줌.

    # 데이터 프레임에 합친 뒤 유사도가 큰 순서로 정렬
    sim = similarity.reshape(-1)
    df['similarity'] = sim

    df_final = df.sort_values(by = 'similarity', ascending = False)

    return df_final

# 단어 벡터화

In [ ]:
# 너무 당연한 재료는 빼는 함수 구현하기
# 텍스트 전처리

In [ ]:
class recipe_recommender:
    def __init__(my_ingredients):
        self.my_ingredients = my_ingredients

        return my_ingredients

In [ ]:
### 알고리즘 클레스로 구현하기

my_ingredients = '라면, 감자, 소금, 버터, 설탕, 고추장, 밥, 후추, 간장, 당근, 소면, 계란'

df2 = find_sim_recipe(df, my_ingredients)
# 여기 부터 조회수 및 조리시간 필터링을 사용한 추천
recommandation = df2.head(50)
# df2.sort_values(by='Hit_num', ascending = False)


# 쿠킹 타임 이내 필터링 하여 hit 높은 순으로 반환
cooking_time = 30 # 희망 조리 시간: 분 단위로 입력
recommandation = recommandation[recommandation['Cooking_time(min)'] <= cooking_time]
# recommandation = recommandation.sort_values(by='Hit_num', ascending = False)
recommandation.head(10)

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# recipe_ingre_list = list(df['Ingredients'])
# # 두 재료 목록 합치기
# doc_list = [my_ingredients] + recipe_ingre_list

# # 벡터화
# vect = CountVectorizer(lowercase=False)
# ingredients_mat = vect.fit_transform(doc_list)
# print(ingredients_mat.shape)

# # 벡터화 한 결과는 희소 행렬이므로 밀집 행렬로 변환하면 단어 목록을 볼 수 있음
# feature_vect_dense = ingredients_mat.todense()

# print(feature_vect_dense)
# print(vect.vocabulary_) # 단어 모음 볼 수 있는 코드

# ### Todo! vocabulary 보니까 '익은' 뭐 이런 것도 벡터화 되어있음. 전처리 해줘야함

# 중요한 코드
- 근데 왜 내 재료 기준으로 백터화 하든 둘다 기준으로 벡터화 하든 유사도는 동일하지? 안 동일 다시 해보니까..

In [ ]:
# 사이킷런 라이브러리 활용 코사인 유사도 계산
# 라이브러리는 먼저 넣는 값을 기준으로 유사도를 계산한다. (따라서 벡터화 할 때 굳이 내 재료만 벡터화 fit할 필요 없음)

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ingredients_mat[0], ingredients_mat[1:]) # 내 재료, 레시피 재료
# print(similarity) # 모든 레시피의 유사도를 다 보여줌.

# 데이터 프레임에 합친 뒤 유사도가 큰 순서로 정렬하면...
sim = similarity.reshape(-1)

print(similarity.shape)
print(similarity[:2])

df['similarity'] = sim

df_final = df.sort_values(by = 'similarity', ascending = False)
df_final.head(10)
# Todo : 이거 유사도 순서로 보여주는 대시보드 있으면 좋을 듯